## Summary

---

## Imports

In [1]:
import concurrent.futures
import contextlib
import gzip
import itertools
import logging
import os
import pickle
import re
import socket
import subprocess
import tempfile
import time
import urllib.request
from pathlib import Path

import dotenv
import more_itertools
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import synapseclient
import synapseutils
from kmbio import PDB
from kmtools import structure_tools
from tqdm.auto import tqdm

## Parameters

In [2]:
NOTEBOOK_DIR = Path("30_humsavar").resolve()
NOTEBOOK_DIR.mkdir(exist_ok=True)

NOTEBOOK_DIR

PosixPath('/home/kimlab5/strokach/workspace/elaspic/elaspic2-cagi6/notebooks/30_humsavar')

In [3]:
if "scinet" in socket.gethostname():
    CPU_COUNT = 40
else:
    CPU_COUNT = max(1, len(os.sched_getaffinity(0)))

CPU_COUNT

32

## Download data

In [4]:
if not NOTEBOOK_DIR.joinpath("humsavar.txt").is_file():
    urllib.request.urlretrieve(
        "https://www.uniprot.org/docs/humsavar.txt", NOTEBOOK_DIR.joinpath("humsavar.txt")
    )

In [5]:
# !head {NOTEBOOK_DIR.joinpath("humsavar.txt")} -n 100

In [6]:
# !grep 'p.Ile34306As' {NOTEBOOK_DIR.joinpath("humsavar.txt")}

## Load data

In [7]:
@contextlib.contextmanager
def tracker(df):
    num_rows = len(df)

    def wrapped(df):
        if len(df) != num_rows:
            print(f"Lost {num_rows - len(df)} rows!")
        return df

    yield wrapped

### `humsavar_df`

In [8]:
rows = []
with NOTEBOOK_DIR.joinpath("humsavar.txt").open("rt") as fin:
    for line in itertools.islice(fin, 44, None):
        line = " ".join(line.split())
        row = line.split(" ", 6)
        if len(row[0]) > 10:
            row = (row[0][:10], row[0][10:], *row[1:-2], " ".join(row[-2:]))
        rows.append(row)

humsavar_df = pd.DataFrame(
    rows[:-5],
    columns=["gene_name", "uniprot_id", "ft_id", "aa_change", "effect", "dbSNP", "disease"],
).replace("-", np.nan)

len(humsavar_df)

79745

In [9]:
humsavar_df = (
    #
    humsavar_df
    # Keep only one row per mutation (ignore multiple diseases)
    .drop("disease", axis=1).drop_duplicates()
    # Remove rows with conflicting effects
    .drop_duplicates(["uniprot_id", "aa_change"], keep=False)
)

len(humsavar_df)

78711

In [10]:
display(humsavar_df.head(4))
display(humsavar_df.tail(4))

,gene_name,uniprot_id,ft_id,aa_change,effect,dbSNP
0,A1BG,P04217,VAR_018369,p.His52Arg,LB/B,rs893184
1,A1BG,P04217,VAR_018370,p.His395Arg,LB/B,rs2241788
2,A1CF,Q9NQ94,VAR_052201,p.Val555Met,LB/B,rs9073
3,A1CF,Q9NQ94,VAR_059821,p.Ala558Ser,LB/B,rs11817448


,gene_name,uniprot_id,ft_id,aa_change,effect,dbSNP
79741,NaN,Q96M66,VAR_039178,p.Arg37His,LB/B,rs350229
79742,NaN,Q96M66,VAR_039179,p.Arg171Ser,LB/B,rs11648228
79743,NaN,Q9N2K0,VAR_017799,p.Val81Leu,LB/B,NaN
79744,NaN,Q9N2K0,VAR_017800,p.Phe150Leu,LB/B,NaN


In [11]:
def format_mutation(aa_change):
    match = re.match("P\.([A-Z]+)([0-9]+)([A-Z]+)", aa_change.upper())
    try:
        return f"{structure_tools.AAA_DICT[match[1]]}{match[2]}{structure_tools.AAA_DICT[match[3]]}"
    except Exception as error:
        print(aa_change)
        return

In [12]:
humsavar_df["mutation"] = humsavar_df["aa_change"].apply(format_mutation)

p.Sec462Gly


In [13]:
with tracker(humsavar_df) as t:
    humsavar_df = t(humsavar_df[humsavar_df["mutation"].notnull()])

Lost 1 rows!


In [14]:
humsavar_df.head(2)

,gene_name,uniprot_id,ft_id,aa_change,effect,dbSNP,mutation
0,A1BG,P04217,VAR_018369,p.His52Arg,LB/B,rs893184,H52R
1,A1BG,P04217,VAR_018370,p.His395Arg,LB/B,rs2241788,H395R


### Already calculated

In [15]:
pfile = pq.ParquetFile(
    NOTEBOOK_DIR.parent.joinpath("30_cagi6_sherloc", "input-data-gby-protein.parquet")
)

In [16]:
already_calculated = set()
for row_group_idx in tqdm(range(pfile.num_row_groups)):
    input_df = pfile.read_row_group(row_group_idx, columns=["protein_id", "mutation"]).to_pandas(
        integer_object_nulls=True
    )
    assert len(input_df) == 1
    row = input_df.iloc[0]
    for mutation in row["mutation"]:
        already_calculated.add((row["protein_id"], mutation))

len(already_calculated)

  0%|          | 0/4182 [00:00<?, ?it/s]

216267

In [17]:
with tracker(humsavar_df) as t:
    humsavar_df = t(
        humsavar_df[
            ~humsavar_df[["uniprot_id", "mutation"]].apply(tuple, axis=1).isin(already_calculated)
        ]
    )

Lost 12236 rows!


## Download sequences (`uniprot_sequences_df`)

In [18]:
def download_sequence(uniprot_id):
    if uniprot_id in ["Q9Y2G2"]:
        uniprot_id = f"{uniprot_id}-1"

    with urllib.request.urlopen(f"https://www.uniprot.org/uniprot/{uniprot_id}.fasta") as f:
        data = f.read().decode("utf-8")
    chunks = []
    for line in data.split("\n"):
        if line.startswith(">"):
            continue
        chunks.append(line.strip())
    sequence = "".join(chunks)

    if uniprot_id == "P11586":
        sequence_lst = list(sequence)
        assert sequence_lst[133] == "K"
        sequence_lst[133] = "R"
        sequence = "".join(sequence_lst)

    return sequence


download_sequence("Q96NU1")

'MSKGILQVHPPICDCPGCRISSPVNRGRLADKRTVALPAARNLKKERTPSFSASDGDSDGSGPTCGRRPGLKQEDGPHIRIMKRRVHTHWDVNISFREASCSQDGNLPTLISSVHRSRHLVMPEHQSRCEFQRGSLEIGLRPAGDLLGKRLGRSPRISSDCFSEKRARSESPQEALLLPRELGPSMAPEDHYRRLVSALSEASTFEDPQRLYHLGLPSHGEDPPWHDPPHHLPSHDLLRVRQEVAAAALRGPSGLEAHLPSSTAGQRRKQGLAQHREGAAPAAAPSFSERELPQPPPLLSPQNAPHVALGPHLRPPFLGVPSALCQTPGYGFLPPAQAEMFAWQQELLRKQNLARLELPADLLRQKELESARPQLLAPETALRPNDGAEELQRRGALLVLNHGAAPLLALPPQGPPGSGPPTPSRDSARRAPRKGGPGPASARPSESKEMTGARLWAQDGSEDEPPKDSDGEDPETAAVGCRGPTPGQAPAGGAGAEGKGLFPGSTLPLGFPYAVSPYFHTGAVGGLSMDGEEAPAPEDVTKWTVDDVCSFVGGLSGCGEYTRVFREQGIDGETLPLLTEEHLLTNMGLKLGPALKIRAQVARRLGRVFYVASFPVALPLQPPTLRAPERELGTGEQPLSPTTATSPYGGGHALAGQTSPKQENGTLALLPGAPDPSQPLC'

In [19]:
uniprot_sequences_file = NOTEBOOK_DIR.joinpath("uniprot-sequences.parquet")

if uniprot_sequences_file.is_file():
    uniprot_sequences_df = pq.read_table(uniprot_sequences_file).to_pandas()
else:
    with concurrent.futures.ThreadPoolExecutor(20) as pool:
        uniprot_ids = humsavar_df["uniprot_id"].unique()
        sequences = list(tqdm(pool.map(download_sequence, uniprot_ids), total=len(uniprot_ids)))
        uniprot_sequences_df = pd.DataFrame({"uniprot_id": uniprot_ids, "sequence": sequences})
    pq.write_table(
        pa.Table.from_pandas(uniprot_sequences_df, preserve_index=False), uniprot_sequences_file
    )

In [20]:
assert uniprot_sequences_df["sequence"].notnull().all()
assert (uniprot_sequences_df["sequence"] != "").all()

In [21]:
uniprot_sequence_lookup = (
    uniprot_sequences_df[["uniprot_id", "sequence"]]
    .drop_duplicates()
    .set_index("uniprot_id")["sequence"]
    .to_dict()
)

list(uniprot_sequence_lookup)[:3]

['P04217', 'Q9NQ94', 'P01023']

## Download structures (`uniprot_structures_df`)

In [22]:
def download_structure(uniprot_id):
    try:
        with urllib.request.urlopen(
            f"https://alphafold.ebi.ac.uk/files/AF-{uniprot_id}-F1-model_v1.pdb"
        ) as f:
            data = f.read().decode("utf-8")
    except urllib.error.HTTPError as error:
        if error.code == 404:
            return None
        raise
    return data


download_structure("Q96NU1")[:20]

'HEADER              '

In [23]:
download_structure("asdf")

In [24]:
uniprot_structures_file = NOTEBOOK_DIR.joinpath("uniprot-structures.parquet")

if uniprot_structures_file.is_file():
    uniprot_structures_df = pq.read_table(uniprot_structures_file).to_pandas()
else:
    with concurrent.futures.ThreadPoolExecutor(20) as pool:
        uniprot_ids = humsavar_df["uniprot_id"].unique()
        structures = list(tqdm(pool.map(download_structure, uniprot_ids), total=len(uniprot_ids)))
        uniprot_structures_df = pd.DataFrame({"uniprot_id": uniprot_ids, "structure": structures})
    pq.write_table(
        pa.Table.from_pandas(uniprot_structures_df, preserve_index=False), uniprot_structures_file
    )

In [25]:
with tracker(uniprot_structures_df) as t:
    uniprot_structures_df = t(uniprot_structures_df[uniprot_structures_df["structure"].notnull()])

Lost 237 rows!


In [26]:
uniprot_structure_lookup = (
    uniprot_structures_df[["uniprot_id", "structure"]]
    .drop_duplicates()
    .set_index("uniprot_id")["structure"]
    .to_dict()
)

list(uniprot_structure_lookup)[:3]

['P04217', 'Q9NQ94', 'P01023']

## Download alignments (`uniprot_alignments_df`)

In [27]:
mmseqs2_output_path = NOTEBOOK_DIR.joinpath("mmseqs2")
mmseqs2_output_path.mkdir(exist_ok=True)

mmseqs2_output_path

PosixPath('/home/kimlab5/strokach/workspace/elaspic/elaspic2-cagi6/notebooks/30_humsavar/mmseqs2')

In [28]:
def get_alignment(uniprot_id, sequence, gateway):
    while True:
        try:
            alignment = mmseqs2.run_mmseqs2(
                [sequence],
                gateway=gateway,
                proxies={
                    "http": "http://20.47.108.204:8888",
                    "https": "https://152.26.229.86:9443",
                },
            )[0]
            break
        except Exception as e:
            print(e)
            time.sleep(10)
            continue
    if alignment[1] != f"{sequence}\n":
        print("Alignment does not match!")
    result = {"uniprot_id": uniprot_id, "alignment": alignment}

    with gzip.open(mmseqs2_output_path.joinpath(f"{uniprot_id}.pickle.gz"), "wb") as fout:
        pickle.dump(result, fout, pickle.HIGHEST_PROTOCOL)

    return result

In [29]:
def iter_alignments(mmseqs2_output_path):
    for file in tqdm(os.listdir(mmseqs2_output_path)):
        uniprot_id = file.split(".")[0]
        with gzip.open(mmseqs2_output_path.joinpath(file)) as fin:
            try:
                data = pickle.load(fin)
            except Exception as e:
                print(e)
                continue
        assert data["uniprot_id"] == uniprot_id
        assert len(data["alignment"]) > 2
        yield data

In [30]:
uniprot_alignments_file = NOTEBOOK_DIR.joinpath("uniprot-alignments.parquet")

if uniprot_alignments_file.is_file():
    uniprot_alignments_df = pq.read_table(
        uniprot_alignments_file, columns=["uniprot_id"]
    ).to_pandas()
else:
    writer = None
    batch_size = 100
    for alignment_batch in more_itertools.chunked(iter_alignments(mmseqs2_output_path), batch_size):
        alignment_batch_df = pd.DataFrame(alignment_batch)
        table = pa.Table.from_pandas(alignment_batch_df, preserve_index=False)
        if writer is None:
            writer = pq.ParquetWriter(uniprot_alignments_file, table.schema, compression="zstd")
        writer.write_table(table)
    writer.close()
    uniprot_alignments_df = pq.read_table(
        uniprot_alignments_file, columns=["uniprot_id"]
    ).to_pandas()

uniprot_alignments_df["have_alignment"] = True

In [31]:
uniprot_alignment_lookup = (
    uniprot_alignments_df[["uniprot_id", "have_alignment"]]
    .drop_duplicates()
    .set_index("uniprot_id")["have_alignment"]
    .to_dict()
)

print(len(uniprot_alignment_lookup))
print(list(uniprot_alignment_lookup)[:3])

11634
['Q96M63', 'Q8NFU5', 'Q5F1R6']


In [32]:
uniprot_sequences_for_aln_df = uniprot_sequences_df[
    ~uniprot_sequences_df["uniprot_id"].isin(uniprot_alignments_df["uniprot_id"].values)
]

uniprot_sequences_for_aln_df["sequence_length"] = uniprot_sequences_for_aln_df["sequence"].str.len()
uniprot_sequences_for_aln_df = uniprot_sequences_for_aln_df.sort_values(
    "sequence_length", ascending=False  # Was True
)

display(uniprot_sequences_for_aln_df)
print(len(uniprot_sequences_for_aln_df))

,uniprot_id,sequence,sequence_length
11726,Q8WZ42,MTTQAPTFTQPLQSVVVLEGSTATFEAHISGFPVPEVSWFRDGQVI...,34350
6803,Q8WXI7,MLKPSGLPGSSSPTRSLMTGSRSTKATPEMDSGLTGATLSPKTSTG...,14507
10786,Q8NF91,MATSRGASRCPRDIANVMQRLQDEQEIVQKRTFTKWINSHLAKRKP...,8797
6805,Q7Z5P9,MKLILWYLVVALWCFFKDVEALLYRQKSDGKIAASRSGGFSYGSSS...,8384
7449,Q5VST9,MDQPQFSGAPRFLTRPKAFVVSVGKDATLSCQIVGNPTPQVSWEKD...,7968
...,...,...,...
7305,Q14995,MEVNAGGVIAYISSSSSASSPASCHSEGSENSFQSSSSSVPSSPNS...,579
155,Q6NUN0,MRPWLRHLVLQALRNSRAFCGSHGKPAPLPVPQKIVATWEAISLGR...,579
9231,Q14699,MGCGLNKLEKRDEKRPGNIYSTLKRPQVETKIDVSYEYRFLEFTTL...,578
2527,P50461,MPNWGGGAKCGACEKTVYHAEEIQCNGRSFHKTCFHCMACRKALDS...,194


1164


In [33]:
# if len(uniprot_sequences_for_aln_df) > 2:
#     from elaspic2.plugins.alphafold import mmseqs2

#     dotenv.load_dotenv("../.env")
#     with mmseqs2.api_gateway(mmseqs2.MMSEQS2_HOST_URL) as gateway:
#         concurrency = 64
#         with concurrent.futures.ThreadPoolExecutor(concurrency) as pool:
#             alignments = list(
#                 tqdm(
#                     pool.map(
#                         get_alignment,
#                         uniprot_sequences_for_aln_df["uniprot_id"],
#                         uniprot_sequences_for_aln_df["sequence"],
#                         itertools.repeat(gateway),
#                     ),
#                     total=len(uniprot_sequences_for_aln_df),
#                 )
#             )

## Validations

### Sanity checks

In [34]:
assert len(humsavar_df) == len(humsavar_df.drop_duplicates(["uniprot_id", "mutation"]))

### Mutation matches sequence

In [35]:
def mutation_matches_sequence(mutation, sequence):
    wt, pos, mut = mutation[0], mutation[1:-1], mutation[-1]
    try:
        pos = int(pos)
    except Exception as e:
        return False
    if pos > len(sequence):
        return False
    return sequence[pos - 1] == wt


mutation_matches_sequence("A5C", "XXXXA")

True

In [36]:
assert not set(uniprot_structure_lookup) - set(uniprot_sequence_lookup)

In [37]:
matches = np.array(
    [
        mutation_matches_sequence(mutation, uniprot_sequence_lookup[uniprot_id])
        for uniprot_id, mutation in humsavar_df[["uniprot_id", "mutation"]].values
    ]
)

humsavar_df[~matches]

,gene_name,uniprot_id,ft_id,aa_change,effect,dbSNP,mutation
10466,CARD8,Q9Y2G2,VAR_048606,p.Ile173Val,LB/B,rs11881179,I173V
10467,CARD8,Q9Y2G2,VAR_061079,p.Glu204Ala,LB/B,rs59878320,E204A
10468,CARD8,Q9Y2G2,VAR_084560,p.Val44Ile,US,NaN,V44I
10469,CARD8,Q9Y2G2,VAR_084561,p.Phe102Ile,LP/P,rs2043211,F102I
44891,MTHFD1,P11586,VAR_016232,p.Lys134Arg,LB/B,rs1950902,K134R


In [38]:
with tracker(humsavar_df) as t:
    humsavar_df = t(humsavar_df[matches])

Lost 5 rows!


### Sequence matches structure

In [39]:
def load_structure_blob(structure_blob):
    with tempfile.NamedTemporaryFile(suffix=".pdb") as tmp_file:
        with open(tmp_file.name, "wt") as fout:
            fout.write(structure_blob)
        structure = PDB.load(tmp_file.name)
    return structure


def sequence_matches_structure(sequence, structure_blob):
    structure = load_structure_blob(structure_blob)
    sequence_from_structure = structure_tools.get_chain_sequence(structure[0]["A"])
    return sequence == sequence_from_structure


sequence_matches_structure("", uniprot_structure_lookup["P04217"])

False

In [40]:
assert not set(uniprot_structure_lookup) - set(uniprot_sequence_lookup)

In [41]:
def worker(row):
    uniprot_id, sequence, structure_blob = row
    if not sequence_matches_structure(sequence, structure_blob):
        print(f"Error: sequence does not match structure for {uniprot_id=}!")
        return uniprot_id
    else:
        return None


with concurrent.futures.ProcessPoolExecutor() as pool:
    mismatches = list(
        tqdm(
            pool.map(
                worker,
                (
                    (up, uniprot_sequence_lookup[up], uniprot_structure_lookup[up])
                    for up in uniprot_structure_lookup
                ),
                chunksize=100,
            ),
            total=len(uniprot_structure_lookup),
        )
    )

mismatches = [m for m in mismatches if m is not None]
mismatches

  0%|          | 0/12561 [00:00<?, ?it/s]

Error: sequence does not match structure for uniprot_id='Q8WYJ6'!


['Q8WYJ6']

In [42]:
with tracker(humsavar_df) as t:
    humsavar_df = t(humsavar_df[~humsavar_df["uniprot_id"].isin(mismatches)])

Lost 1 rows!


### Have sequence, structure, and alignment

In [43]:
with tracker(humsavar_df) as t:
    humsavar_df = t(humsavar_df[humsavar_df["uniprot_id"].isin(uniprot_sequence_lookup)])

In [44]:
with tracker(humsavar_df) as t:
    humsavar_df = t(humsavar_df[humsavar_df["uniprot_id"].isin(uniprot_structure_lookup)])  # 5289

Lost 5289 rows!


In [45]:
# with tracker(humsavar_df) as t:
#     humsavar_df = t(humsavar_df[humsavar_df["uniprot_id"].isin(uniprot_alignment_lookup)])  # 9524

### Sequence matches alignment

In [46]:
# for uniprot_id, sequence, alignment in tqdm(
#     final_df[["uniprot_id", "sequence", "alignment"]].values
# ):
#     assert sequence == alignment[1].strip()

## Write results

In [47]:
humsavar_df["mutation_pos"] = humsavar_df["mutation"].str[1:-1].astype(int)

humsavar_df = humsavar_df.sort_values(["uniprot_id", "mutation_pos"])

humsavar_df.head(2)

,gene_name,uniprot_id,ft_id,aa_change,effect,dbSNP,mutation,mutation_pos
18104,CYP2D7,A0A087X1C5,VAR_072632,p.Ser70Asn,LB/B,rs11090077,S70N,70
18105,CYP2D7,A0A087X1C5,VAR_072633,p.Ser311Leu,LB/B,rs1800754,S311L,311


In [48]:
def load_alignments(uniprot_ids_and_sequences):
    alignments = {}
    for uniprot_id, sequence in uniprot_ids_and_sequences:
        mmseqs2_file = mmseqs2_output_path.joinpath(f"{uniprot_id}.pickle.gz")
        if not mmseqs2_file.is_file():
            continue
        with gzip.open(mmseqs2_file) as fin:
            data = pickle.load(fin)
        assert data["uniprot_id"] == uniprot_id
        assert len(data["alignment"]) > 2
        assert sequence == data["alignment"][1].strip()
        alignments[uniprot_id] = data["alignment"]
    return alignments

In [49]:
def write_table(df, output_file, schema, writer):
    if schema is None:
        table = pa.Table.from_pandas(df, preserve_index=False)
        schema = table.schema
    else:
        table = pa.Table.from_pandas(df, schema=schema, preserve_index=False)
    if writer is None:
        writer = pq.ParquetWriter(output_file, schema)
    writer.write_table(table)
    return schema, writer

In [50]:
output_file_1 = NOTEBOOK_DIR.joinpath("humsavar-gby-protein.parquet")
output_file_2 = NOTEBOOK_DIR.joinpath("humsavar-gby-protein-waln.parquet")

group_size = 100

schema_1 = None
schema_2 = None
writer_1 = None
writer_2 = None
for start in tqdm(range(0, len(humsavar_df), group_size)):
    df = (
        humsavar_df.iloc[start : start + group_size]
        .groupby(["uniprot_id"], dropna=False)
        .agg({"mutation": list, "effect": list})
        .reset_index()
        .merge(
            uniprot_sequences_df,
            on=["uniprot_id"],
        )
        .merge(
            uniprot_structures_df,
            on=["uniprot_id"],
        )
    )
    schema_1, writer_1 = write_table(df, output_file_1, schema_1, writer_1)

    alignments = load_alignments(df[["uniprot_id", "sequence"]].drop_duplicates().values)
    df["alignment"] = df["uniprot_id"].map(alignments)
    df = df[df["alignment"].notnull()]

    if df.empty:
        continue

    schema_2, writer_2 = write_table(df, output_file_2, schema_2, writer_2)

writer_1.close()
writer_2.close()

output_file_1, output_file_2

  0%|          | 0/612 [00:00<?, ?it/s]

(PosixPath('/home/kimlab5/strokach/workspace/elaspic/elaspic2-cagi6/notebooks/30_humsavar/humsavar-gby-protein.parquet'),
 PosixPath('/home/kimlab5/strokach/workspace/elaspic/elaspic2-cagi6/notebooks/30_humsavar/humsavar-gby-protein-waln.parquet'))

In [51]:
pfile_1 = pq.ParquetFile(output_file_1)
pfile_2 = pq.ParquetFile(output_file_2)

pfile_1.num_row_groups, pfile_2.num_row_groups

(612, 599)

In [52]:
pfile_1.metadata

  created_by: parquet-cpp-arrow version 5.0.0
  num_columns: 5
  num_rows: 13050
  num_row_groups: 612
  format_version: 1.0
  serialized_size: 1065992

In [53]:
pfile_2.metadata

  created_by: parquet-cpp-arrow version 5.0.0
  num_columns: 6
  num_rows: 12097
  num_row_groups: 599
  format_version: 1.0
  serialized_size: 1279208